# An Introduction to Zygote: Linear Regression

In this notebook, we will define Linear Regression in Zygote from scratch, showing how easy it is to take derivatives of custom code.

In [1]:
# Initialize environment in current directory, to load
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()
using Zygote, LinearAlgebra

┌ Info: activating environment at `~/src/msr_talk/Project.toml`.
└ @ Pkg.API /Users/sabae/tmp/julia-build/julia-release-1.2/usr/share/julia/stdlib/v1.2/Pkg/src/API.jl:564


  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %

┌ Info: Recompiling stale cache file /Users/sabae/.julia/compiled/v1.2/Zygote/4kbLI.ji for Zygote [e88e6eb3-aa80-5325-afca-941959d7151f]
└ @ Base loading.jl:1240


This example will showcase how we do a simple linear fit with Zygote, making use of complex datastructures, a home-grown stochastic gradient descent optimizer, and some good old-fashioned math.  We start with the problem
statement:  We wish to learn the mapping `f(X) -> Y`, where `X` is a matrix of vector observations, `f()` is a linear mapping function and `Y` is a vector of scalar observations.

Because we like complex objects, we will define our linear regression as the following object:

In [4]:
# LinearRegression object, containing multiple fields, some of which will be learned.
mutable struct LinearRegression
    # These values will be implicitly learned
    weights::Matrix
    bias::Float64

    # These values will not be learned
    name::String
end

LinearRegression(nparams, name) = LinearRegression(randn(1, nparams), 0.0, name)

LinearRegression

We will define two verbs to act upon a `LinearRegression` object; `predict()`, to perform the linear regression, and `loss()` to measure the $\ell_2$ norm between a target and our current prediction.

In [5]:
# Our linear regression looks very familiar; w*X + b
function predict(model::LinearRegression, X)
    return model.weights * X .+ model.bias
end

# Our "loss" that must be minimized is the l2 norm between our current
# prediction and our ground-truth Y
function loss(model::LinearRegression, X, Y)
    return norm(predict(model, X) .- Y, 2)
end


loss (generic function with 1 method)

In [6]:
# Our "ground truth" values (that we will learn, to prove that this works)
weights_gt = [1.0, 2.7, 0.3, 1.2]'
bias_gt = 0.4

# Generate a dataset of many observations
X = randn(length(weights_gt), 10000)
Y = weights_gt * X .+ bias_gt

# Add a little bit of noise to `X` so that we do not have an exact solution,
# but must instead do a least-squares fit:
X .+= 0.001.*randn(size(X))

4×10000 Array{Float64,2}:
 -1.19203    1.91616     0.843466  …  -1.57857    0.246021  -0.597997
 -0.48234    1.32798    -0.899407      0.043473   1.44814   -0.89648 
  1.55866    0.0718622   1.89751       0.994072   0.638415  -0.273722
 -0.558112  -1.68074     0.954623      0.442166  -0.187531  -0.92237 

In [8]:
# Now we begin our "training loop", where we take examples from `X`,
# calculate loss with respect to the corresponding entry in `Y`, find the
# gradient upon our model, update the model, and continue.  Before we jump
# in, let's look at what `Zygote.gradient()` gives us:
model = LinearRegression(size(X, 1), "Example")

# Calculate gradient upon `model` for the first example in our training set
grads = Zygote.gradient(model) do m
    return loss(m, X[:,1], Y[1])
end

(Base.RefValue{Any}((weights = [-1.192026114794742 -0.482340240894484 1.5586551748882012 -0.5581118308574825], bias = 1.0, name = nothing)),)

The `grads` object is a Tuple containing one element per argument to `gradient()`, so we take the first one to get the gradient upon `model`:

In [9]:
grads = grads[1]

Base.RefValue{Any}((weights = [-1.192026114794742 -0.482340240894484 1.5586551748882012 -0.5581118308574825], bias = 1.0, name = nothing))

Because our LinearRegression object is mutable, the gradient holds a reference to it, which we peel via `grads[]`:


In [10]:
grads = grads[]

(weights = [-1.192026114794742 -0.482340240894484 1.5586551748882012 -0.5581118308574825], bias = 1.0, name = nothing)

We now get a `NamedTuple` so we can now do things like `grads.weights`. Note that while `weights` and `bias` have gradients, `name` just naturally has a  gradient of `nothing`, because it was not involved in the calculation of the output loss.

In [11]:
grads.weights

1×4 Array{Float64,2}:
 -1.19203  -0.48234  1.55866  -0.558112

Next, we will define an update rule that will allow us to modify the weights of our model according to the gradients, using the simplest gradient descent update rule.  We'll then run a training loop to update our weights with the loss from the training set, as we would expect:

In [12]:
# Let's define 
function sgd_update!(model::LinearRegression, grads, η = 0.001)
    model.weights .-= η .* grads.weights
    model.bias -= η * grads.bias
end

sgd_update! (generic function with 2 methods)

In [13]:
# Now let's do that for each example in our training set:
@info("Running train loop for $(size(X,2)) iterations")
for idx in 1:size(X, 2)
    grads = Zygote.gradient(m -> loss(m, X[:, idx], Y[idx]), model)[1][]
    sgd_update!(model, grads)
end

┌ Info: Running train loop for 10000 iterations
└ @ Main In[13]:2


In [14]:
weights_gt

1×4 Adjoint{Float64,Array{Float64,1}}:
 1.0  2.7  0.3  1.2

In [15]:
model.weights

1×4 Array{Float64,2}:
 1.00142  2.70157  0.300252  1.20033

In [16]:
bias_gt

0.4

In [17]:
model.bias

0.3980000000000003